In [122]:
import math
from __future__ import division
from collections import OrderedDict
import histbook as hb
import uproot, uproot_methods

f = uproot.open("nano_95.root")
events = f["Events"]

In [123]:
import fnal_column_analysis_tools.lookup_tools as lookup_tools
import cloudpickle
import zlib

# import a bunch of correction histograms
weightsext = lookup_tools.extractor()
correctionDescriptions = open("newCorrectionFiles.txt").readlines()
weightsext.add_weight_sets(correctionDescriptions)
weightsext.finalize()
weights_eval = weightsext.make_evaluator()
#let's pickle and zip it
weights_names = zlib.compress(cloudpickle.dumps(weightsext._names))
weights_vals = zlib.compress(cloudpickle.dumps(weightsext._weights))

#dir(weights_eval)
#print(weights_eval["muScaleFactor_TightId_Iso"])

In [124]:
all_hists = OrderedDict()
cat = hb.groupby("category", keeporder=True)

def add_1d(*args,**kwargs):
    h = None 
    if 'wgt' in kwargs:
        h = hb.Hist(hb.bin(*args), cat, weight=kwargs['wgt'])
    else:
        h = hb.Hist(hb.bin(*args), cat)    
    all_hists[args[0]] = h
    return h

add_1d("FatJetPt", 500, 0, 1000)
add_1d("MuonPt", 500, 0, 1000)
add_1d("ElectronPt", 500, 0, 1000)
add_1d("MuonE", 500, 0, 1000)
add_1d("ElectronE", 500, 0, 1000)
add_1d("MuonEta", 500, 0, 1000)
add_1d("ElectronEta", 500, 0, 1000)
add_1d("stripeThroughput", 300, 0, 1000000)
add_1d("FatJetCounts",15,0,15)
add_1d("MET",500,0,1000)
#add_1d("stripeTime", 1000, 0.0, 0.5)
#add_1d("leadingLeptonPt", 100, 0, 500,wgt="weight")
#add_1d("zMass", 120, 60, 180,wgt="weight")

display_opts = {'width': 300, 'height': 300}


In [125]:
import numpy as np
import awkward
import cloudpickle
import zlib
import time
from collections import OrderedDict
import fnal_column_analysis_tools
from fnal_column_analysis_tools.analysis_objects import JaggedCandidateArray
from fnal_column_analysis_tools.striped import ColumnGroup, PhysicalColumnGroup, jaggedFromColumnGroup
from fnal_column_analysis_tools.striped.WorkerAddons import Timer
from fnal_column_analysis_tools.lookup_tools import evaluator

for key in events.keys():
    if key.startswith('Jet'):
       print key


#I believe when running in striped, you would replaces events.array["Jet_VarName"].content with events.Jet.VarName 
jet_cands = JaggedCandidateArray.candidatesfromoffsets(events.array("Jet_pt").offsets,
                                                       pt=events.array("Jet_pt").content,
                                                       eta=events.array("Jet_eta").content,
                                                       phi=events.array("Jet_phi").content,
                                                       mass=events.array("Jet_mass").content,
                                                       jetId = events.array("Jet_jetId").content
                                                      )

jet_selection = (jet_cands.pt > 100) * (abs(jet_cands.eta) < 2.4) * (jet_cands.jetId > 0)
selected_jets = jet_cands[jet_selection]

#Fat Jet selection
fatJet_cands = JaggedCandidateArray.candidatesfromoffsets(events.array("AK15Puppi_pt").offsets,
                                                          pt=events.array("AK15Puppi_pt").content,
                                                          eta=events.array("AK15Puppi_eta").content,
                                                          phi=events.array("AK15Puppi_phi").content,
                                                          mass=events.array("AK15Puppi_mass").content,
                                                          jetId = events.array("AK15Puppi_jetId").content
                                                          )
fatJet_selection = (fatJet_cands.pt > 200) * (fatJet_cands.jetId > 0)
selected_fatJets = fatJet_cands[fatJet_selection]

#Tau selection
tau_cands = JaggedCandidateArray.candidatesfromoffsets(events.array("Tau_pt").offsets,
                                                       pt=events.array("Tau_pt").content,
                                                       eta=events.array("Tau_eta").content,
                                                       phi=events.array("Tau_phi").content,
                                                       mass=events.array("Tau_mass").content
                                                      )

#Electron selection
electrons = JaggedCandidateArray.candidatesfromoffsets(events.array("Electron_pt").offsets,
                                                       pt=events.array("Electron_pt").content,
                                                       eta=events.array("Electron_eta").content,
                                                       phi=events.array("Electron_phi").content,
                                                       mass=events.array("Electron_mass").content,
                                                       dxy=events.array("Electron_dxy").content,
                                                       dz=events.array("Electron_dz").content,
                                                       iso=events.array("Electron_pfRelIso03_all").content,
                                                       mva=events.array("Electron_mvaFall17Iso_WP90").content
                                                      )
ele_selection = (electrons.pt>7)*(abs(electrons.eta)<2.4)*(abs(electrons.dxy)<0.05)*(abs(electrons.dz)<0.2)*(electrons.iso<0.4)*(electrons.mva)
selected_ele = ele_cands[ele_selection]

#Gen particles
gen_cands = JaggedCandidateArray.candidatesfromoffsets(events.array("GenPart_pt").offsets,
                                                       pt=events.array("GenPart_pt").content,
                                                       eta=events.array("GenPart_eta").content,
                                                       phi=events.array("GenPart_phi").content,
                                                       mass=events.array("GenPart_mass").content,
                                                       pdgId=events.array("GenPart_pdgId").content,
                                                       momId=events.array("GenPart_genPartIdxMother").content
                                                      )

#combinations = jet_cands.p4.cross(gen_cands.p4[abs(gen_cands.pdgId)==11],nested=True)
combinations = jet_cands.p4.cross(gen_cands.p4,nested=True)
print combinations.flatten().size
print combinations.flatten().flatten().size
#combinations = selected_ele.cross(gen_cands, nested=True)
#jets[~(delta_r(combinations.i0, combinations.i1) < 0.1).any()]

MET=events.array("MET_pt")

signal_events=(selected_fatJets.counts==1)
#signal_events = events[(selected_fatJets.counts==1) * (electrons.counts==0) * (tau_cands==0) ]


#print fatJet_cands.flatten().size
#print fatJet_selection
#print selected_fatJets.flatten().size
#print np.count_nonzero(signal_events)
#print MET[signal_events].s

Jet_area
Jet_btagCMVA
Jet_btagCSVV2
Jet_btagDeepB
Jet_btagDeepC
Jet_btagDeepFlavB
Jet_chEmEF
Jet_chHEF
Jet_eta
Jet_mass
Jet_muEF
Jet_neEmEF
Jet_neHEF
Jet_phi
Jet_pt
Jet_qgl
Jet_rawFactor
Jet_bRegCorr
Jet_bRegRes
Jet_electronIdx1
Jet_electronIdx2
Jet_jetId
Jet_muonIdx1
Jet_muonIdx2
Jet_nConstituents
Jet_nElectrons
Jet_nMuons
Jet_puId
Jet_genJetIdx
Jet_hadronFlavour
Jet_partonFlavour
Jet_cleanmask
302099
10391790
